In [1]:
import pandas as pd
import numpy as np
import glob
import os
import pickle
from IPython.display import clear_output

In [2]:
soil_dir = '../wheat_all/done/**/EnvData.xls'
yield_dir = '../wheat_all/done/'
weather_file = '../iwin_weather/IWIN_Weather_AgERA5_20210211.txt'
pickle_dump_directory = '../processed_data/'

tr_file = '../processed_data/feature_selected_training_{}.pkl'
test_file = '../processed_data/feature_selected_test_{}.pkl'
test_unique_env_file = '../processed_data/feature_selected_unique_env_{}.pkl'
validation_file = '../processed_data/feature_selected_validation_{}.pkl'

In [3]:
def dump_data_as_pickle(filename, data):
    pickle_dump_directory = '../processed_data/'
    filename = pickle_dump_directory + filename
    
    with open(filename, 'wb') as wfile:
        pickle.dump(data, wfile)

In [4]:
def is_float(x):
    try:
        float(x)
    except ValueError:
        return False
    return True

In [5]:
def calc_average(x):
    x = np.array(x, dtype=float)
    avg = np.average(x)
    
    return avg

In [6]:
def read_soil_env(base_dir):
    
    env_data = pd.DataFrame()
    files = glob.glob(base_dir, recursive = True)
    
    for file in files:
        trial_env_data = pd.read_csv(file, delimiter='\t', encoding = "ISO-8859-1")
        trial_env_data['nursary'] = file.split('/')[-2]
        env_data = env_data.append(trial_env_data, ignore_index = True)
        
    return env_data

In [7]:
def process_soil_env(soil_data):
    columns = ['HARVEST_STARTING_DATE', 'HARVEST_FINISHING_DATE', 
               'SOWING_DATE']
    
    soil_data_traits = soil_data[soil_data['Trait name'].isin(columns)]
    
    soil_data_traits = pd.pivot_table(soil_data_traits, index=['Occ', 'Loc_no', 'nursary'], 
                                      columns=['Trait name'], values='Value',
                                     aggfunc={
                                         'Value': lambda x: x.iloc[0],
                                         
                                     })
    soil_data_traits = soil_data_traits.reset_index()
    return soil_data_traits

In [8]:
def combine_yield_soil(yield_data, soil_data_traits):
    yield_soil = yield_data.merge(soil_data_traits, left_on=['Loc_no', 'trial'], right_on=['Loc_no', 'nursary'], how='inner')
    
    return yield_soil

In [9]:
def read_yield_data(directory):
    
    yield_data = pd.DataFrame()
    with os.scandir(directory) as prime_dir:
        for entry in prime_dir:
            trial_name = entry.name
            yield_file = directory + trial_name + '/GrnYld.xls'
            x = pd.read_csv(yield_file, delimiter='\t')
            x['trial'] = trial_name
            yield_data = yield_data.append(x, ignore_index = True)
            
    yield_data = yield_data[yield_data.Value.apply(lambda x: is_float(x))]
    
    yield_data_groupby = yield_data.groupby(['Cid', 'Sid', 'Loc_no', 'trial', 'Cycle']).agg({
            'Value': lambda x: calc_average(x)
    })
    
    yield_data_groupby.reset_index(inplace=True)
    
    return yield_data_groupby

In [10]:
def read_weather_data(weather_file):
    weather_data = pd.read_csv(weather_file, delimiter=',')
    
    return weather_data

In [11]:
def get_env_by_last_eight_m_avg(combined_data, weather_data):
    locations = combined_data['Loc_no'].unique()
    nursaries = combined_data['nursary'].unique()
    
    final_env_data = pd.DataFrame()
    for location in locations:
        for nursary in nursaries:
            loc_spec_data = combined_data[(combined_data['Loc_no'] == location) & (combined_data['nursary'] == nursary)]
            if loc_spec_data.shape[0] > 30:
                start_date = loc_spec_data['start_date'].iloc[0]
                end_date = loc_spec_data['end_date'].iloc[0]

                partial_weather = weather_data[(weather_data['location']==location)
                                              & (weather_data['date']>=start_date)
                                              & (weather_data['date']<=end_date)]

                if partial_weather.shape[0] > 0:
                    partial_weather_gropuby =partial_weather.groupby(['Month']).agg({
                        'Precipitation [mm]': np.mean,
                        'Relative Humidity max [%]': np.mean, 
                        'Relative Humidity min [%]': np.mean,
                        'Shortwave Radiation [MJ/m2/d]': np.mean,
                        'TemperatureMax [C]': np.mean,
                        'TemperatureMin [C]': np.mean,
                        'Vapor Pressure Deficit max [kPa]': np.mean,
                        'Wind Speed 2m [m/s]': np.mean,
                        'Wind Speed 10m [m/s]': np.mean
                    })

                    partial_weather_flatten = partial_weather_gropuby.to_numpy().flatten().tolist()

                    if len(partial_weather_flatten) < 81:
                        partial_weather_flatten = partial_weather_flatten + [0] * (81 - len(partial_weather_flatten))


                    z = loc_spec_data
                    z['weather'] = [partial_weather_flatten for i in range(loc_spec_data.shape[0])]
                    clear_output(wait=True)
                    final_env_data = final_env_data.append(z)
                    
    return final_env_data


In [12]:
def create_dataset_with_env_data(geno_data, env_data):
    env_data = env_data[['Cid', 'Sid', 'Value', 'weather']]
    geno_data_combined = geno_data.merge(env_data, left_on=['CID', 'SID'], right_on=['Cid', 'Sid'], how='inner')
    
    final_geno = geno_data_combined.drop(['Cid', 'Sid', 'Value', 'weather', 'CID', 'SID', 'GID'], axis =1)
    final_geno = final_geno.to_numpy()
    
    weather = geno_data_combined['weather'].tolist()
    weather = np.array(weather)
    
    final_geno = np.concatenate((final_geno, weather), axis = 1)
    pheno = geno_data_combined['Value'].to_numpy()
    
    return final_geno, pheno

In [13]:
def create_dataset_with_all_env_data(geno_data, env_data):
    geno_data_combined = geno_data.merge(env_data, left_on=['CID', 'SID'], right_on=['Cid', 'Sid'], how='inner')
    
    
    
    return geno_data_combined

In [14]:
soil_data = read_soil_env(soil_dir)
soil_data.head()

,Trial name,Occ,Loc_no,Country,Loc_desc,Cycle,Trait No,Trait name,Value,Unit,nursary
0,17TH HIGH RAINFALL WHEAT YT,2,51003,ARGENTINA,PERGAMINO -EEA - INTA,2009,282,AREA_HARVESTED_BED_PLOT_M2,5,m2,17HRWYT
1,17TH HIGH RAINFALL WHEAT YT,5,50208,BRAZIL,OCEPAR-PALOTINA,2009,282,AREA_HARVESTED_BED_PLOT_M2,6,m2,17HRWYT
2,17TH HIGH RAINFALL WHEAT YT,12,50246,BRAZIL,CNP-SOJA,2009,282,AREA_HARVESTED_BED_PLOT_M2,6,m2,17HRWYT
3,17TH HIGH RAINFALL WHEAT YT,29,51409,PARAGUAY,CENTRO INV. AGR. SEMILLAS,2009,282,AREA_HARVESTED_BED_PLOT_M2,2,m2,17HRWYT
4,17TH HIGH RAINFALL WHEAT YT,36,11103,ZIMBABWE,RATTRAY ARNOLD RES. STN.,2009,282,AREA_HARVESTED_BED_PLOT_M2,6.6,m2,17HRWYT


In [17]:
soil_data[['Loc_no', 'Country']].to_pickle('../processed_data/location_info.pkl')

In [15]:
soil_data_traits = process_soil_env(soil_data)

In [16]:
soil_data_traits.columns

Index(['Occ', 'Loc_no', 'nursary', 'HARVEST_FINISHING_DATE',
       'HARVEST_STARTING_DATE', 'SOWING_DATE'],
      dtype='object', name='Trait name')

In [17]:
soil_data_traits.head()

Trait name,Occ,Loc_no,nursary,HARVEST_FINISHING_DATE,HARVEST_STARTING_DATE,SOWING_DATE
0,1,10002,1WYCYT,Feb 2 2014,Dec 23 2013,Jul 17 2013
1,1,10009,33ESWYT,Nov 22 2012,Nov 21 2012,Jun 13 2012
2,1,12308,41IBWSN,Oct 22 2008,Oct 22 2008,Jun 11 2008
3,1,12902,18HRWYT,Sep 21 2010,Sep 21 2010,Apr 20 2010
4,1,12902,31ESWYT,NaN,NaN,Apr 20 2010


In [18]:
yield_data = read_yield_data(yield_dir)
yield_data.head()

,Cid,Sid,Loc_no,trial,Cycle,Value
0,1382,1,10002,37IBWSN,2004,3.200
1,1382,1,10026,36IBWSN,2003,7.410
2,1382,1,10029,37IBWSN,2004,4.773
3,1382,1,10032,37IBWSN,2004,2.529
4,1382,1,11011,38IBWSN,2005,1.864


In [19]:
combined_data = combine_yield_soil(yield_data, soil_data_traits)
combined_data['SOWING_DATE'] = pd.to_datetime(combined_data['SOWING_DATE'])
combined_data['HARVEST_FINISHING_DATE'] = pd.to_datetime(combined_data['HARVEST_FINISHING_DATE'])
combined_data.head()

,Cid,Sid,Loc_no,trial,Cycle,Value,Occ,nursary,HARVEST_FINISHING_DATE,HARVEST_STARTING_DATE,SOWING_DATE
0,1382,1,10002,37IBWSN,2004,3.20,9,37IBWSN,NaT,NaN,2004-07-13
1,2565,3,10002,37IBWSN,2004,2.77,9,37IBWSN,NaT,NaN,2004-07-13
2,4746,1,10002,37IBWSN,2004,2.68,9,37IBWSN,NaT,NaN,2004-07-13
3,7896,281,10002,37IBWSN,2004,4.12,9,37IBWSN,NaT,NaN,2004-07-13
4,7896,410,10002,37IBWSN,2004,3.07,9,37IBWSN,NaT,NaN,2004-07-13


In [20]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 187866 entries, 0 to 187865
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Cid                     187866 non-null  int64         
 1   Sid                     187866 non-null  int64         
 2   Loc_no                  187866 non-null  int64         
 3   trial                   187866 non-null  object        
 4   Cycle                   187866 non-null  int64         
 5   Value                   187866 non-null  float64       
 6   Occ                     187866 non-null  int64         
 7   nursary                 187866 non-null  object        
 8   HARVEST_FINISHING_DATE  169040 non-null  datetime64[ns]
 9   HARVEST_STARTING_DATE   180408 non-null  object        
 10  SOWING_DATE             187264 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(5), object(3)
memory usage: 17.2+ MB


In [21]:
combined_data['start_date'] = combined_data.apply(lambda x: (x.SOWING_DATE - pd.DateOffset(months=2)) 
                                                  if pd.isnull(x.SOWING_DATE) == False else x.HARVEST_FINISHING_DATE - pd.DateOffset(months=8), axis=1) 

In [22]:
combined_data.head()

,Cid,Sid,Loc_no,trial,Cycle,Value,Occ,nursary,HARVEST_FINISHING_DATE,HARVEST_STARTING_DATE,SOWING_DATE,start_date
0,1382,1,10002,37IBWSN,2004,3.20,9,37IBWSN,NaT,NaN,2004-07-13,2004-05-13
1,2565,3,10002,37IBWSN,2004,2.77,9,37IBWSN,NaT,NaN,2004-07-13,2004-05-13
2,4746,1,10002,37IBWSN,2004,2.68,9,37IBWSN,NaT,NaN,2004-07-13,2004-05-13
3,7896,281,10002,37IBWSN,2004,4.12,9,37IBWSN,NaT,NaN,2004-07-13,2004-05-13
4,7896,410,10002,37IBWSN,2004,3.07,9,37IBWSN,NaT,NaN,2004-07-13,2004-05-13


In [23]:
combined_data.shape

(187866, 12)

In [24]:
combined_data = combined_data.dropna(axis=0, subset=['start_date'])

In [25]:
combined_data.shape

(187490, 12)

In [26]:
combined_data['end_date'] = combined_data.apply(lambda x: (x.start_date + pd.DateOffset(months=8)), axis =1)

In [27]:
combined_data.head()

,Cid,Sid,Loc_no,trial,Cycle,Value,Occ,nursary,HARVEST_FINISHING_DATE,HARVEST_STARTING_DATE,SOWING_DATE,start_date,end_date
0,1382,1,10002,37IBWSN,2004,3.20,9,37IBWSN,NaT,NaN,2004-07-13,2004-05-13,2005-01-13
1,2565,3,10002,37IBWSN,2004,2.77,9,37IBWSN,NaT,NaN,2004-07-13,2004-05-13,2005-01-13
2,4746,1,10002,37IBWSN,2004,2.68,9,37IBWSN,NaT,NaN,2004-07-13,2004-05-13,2005-01-13
3,7896,281,10002,37IBWSN,2004,4.12,9,37IBWSN,NaT,NaN,2004-07-13,2004-05-13,2005-01-13
4,7896,410,10002,37IBWSN,2004,3.07,9,37IBWSN,NaT,NaN,2004-07-13,2004-05-13,2005-01-13


In [28]:
weather_data = read_weather_data(weather_file)

In [29]:
weather_data.head()

,location,Year,Month,Day,lat,lon,Precipitation [mm],Relative Humidity max [%],Relative Humidity min [%],Shortwave Radiation [MJ/m2/d],TemperatureMax [C],TemperatureMin [C],Vapor Pressure Deficit max [kPa],Wind Speed 2m [m/s],Wind Speed 10m [m/s]
0,10001,1979,1,1,-26.167,26.167,1.6,63.1,42.3,28.2,29.4,17.3,5.6,3.9,5.2
1,10001,1979,1,2,-26.167,26.167,0.1,62.7,39.1,29.0,30.1,17.5,6.6,3.6,4.9
2,10001,1979,1,3,-26.167,26.167,0.0,50.3,28.5,32.5,27.6,13.5,9.2,3.9,5.2
3,10001,1979,1,4,-26.167,26.167,0.2,76.2,56.4,19.3,21.8,14.4,2.0,3.6,4.9
4,10001,1979,1,5,-26.167,26.167,0.7,61.7,41.4,29.3,26.4,13.6,4.9,2.8,3.8


In [30]:
weather_data['date'] = pd.to_datetime(weather_data[['Year', 'Month', 'Day']])

In [31]:
final_env_data = get_env_by_last_eight_m_avg(combined_data, weather_data)

<ipython-input-11-13986c2a7dbc>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  z['weather'] = [partial_weather_flatten for i in range(loc_spec_data.shape[0])]


In [32]:
print(final_env_data.shape)
final_env_data.head()

(177683, 14)


,Cid,Sid,Loc_no,trial,Cycle,Value,Occ,nursary,HARVEST_FINISHING_DATE,HARVEST_STARTING_DATE,SOWING_DATE,start_date,end_date,weather
0,1382,1,10002,37IBWSN,2004,3.20,9,37IBWSN,NaT,NaN,2004-07-13,2004-05-13,2005-01-13,"[7.2846153846153845, 85.62307692307692, 45.746..."
1,2565,3,10002,37IBWSN,2004,2.77,9,37IBWSN,NaT,NaN,2004-07-13,2004-05-13,2005-01-13,"[7.2846153846153845, 85.62307692307692, 45.746..."
2,4746,1,10002,37IBWSN,2004,2.68,9,37IBWSN,NaT,NaN,2004-07-13,2004-05-13,2005-01-13,"[7.2846153846153845, 85.62307692307692, 45.746..."
3,7896,281,10002,37IBWSN,2004,4.12,9,37IBWSN,NaT,NaN,2004-07-13,2004-05-13,2005-01-13,"[7.2846153846153845, 85.62307692307692, 45.746..."
4,7896,410,10002,37IBWSN,2004,3.07,9,37IBWSN,NaT,NaN,2004-07-13,2004-05-13,2005-01-13,"[7.2846153846153845, 85.62307692307692, 45.746..."


In [33]:
versions = ['v1', 'v2', 'v3', 'v4', 'v5']

for version in versions:
    
    with open('../processed_data/train_test_by_env_'+version+'.pkl', 'rb') as outfile:
        train_test_by_env = pickle.load(outfile)
    
    tr_trials = train_test_by_env[(train_test_by_env['training']==True)]['index'].tolist()
    test_trials = train_test_by_env[(train_test_by_env['test']==True) & \
                                    ((train_test_by_env['training']==True) | \
                                     (train_test_by_env['validation']==True))]['index'].tolist()
                                     
    test_trials_unique_env = train_test_by_env[(train_test_by_env['test']==True) & \
                                    ((train_test_by_env['training']==False) | 
                                     (train_test_by_env['validation']==False))]['index'].tolist()
    
    val_trials = train_test_by_env[(train_test_by_env['validation']==True)]['index'].tolist()
    
    
    tr_data = pd.read_pickle(tr_file.format(version))
    print('version: ', version)
    
    tr_final_env = final_env_data[final_env_data['trial'].isin(tr_trials)]
    final_tr, tr_pheno = create_dataset_with_env_data(tr_data,tr_final_env)
    final_tr_with_all_data = create_dataset_with_all_env_data(tr_data, tr_final_env)
    
    print('tr data shape: ', final_tr.shape)
    
    test_data = pd.read_pickle(test_file.format(version))
    test_final_env = final_env_data[final_env_data['trial'].isin(test_trials)]
    final_test, test_pheno = create_dataset_with_env_data(test_data, test_final_env)
    final_test_with_all_data = create_dataset_with_all_env_data(test_data, test_final_env)
    
    print('test data shape: ', final_test.shape)
    
    test_data_unique_env = pd.read_pickle(test_unique_env_file.format(version))
    test_final_env = final_env_data[final_env_data['trial'].isin(test_trials_unique_env)]
    final_test_unique_env, test_pheno_unique_env = create_dataset_with_env_data(test_data_unique_env, test_final_env)
    final_test_with_all_data_unique_env = create_dataset_with_all_env_data(test_data_unique_env, test_final_env)
    
    print('test data unique env shape: ', final_test_unique_env.shape)
    
    val_data = pd.read_pickle(validation_file.format(version))
    val_final_env = final_env_data[final_env_data['trial'].isin(val_trials)]
    final_val, val_pheno = create_dataset_with_env_data(val_data,val_final_env)
    final_val_with_all_data = create_dataset_with_all_env_data(val_data, val_final_env)
    
    print('val data shape: ', final_val.shape)
    
    dump_data_as_pickle('tr_geno_with_8_m_avg_' + version +'.pkl', final_tr)
    dump_data_as_pickle('tr_pheno_with_8_m_avg_' + version + '.pkl', tr_pheno)
    dump_data_as_pickle('tr_pheno_with_8_m_avg_all_data_' + version + '.pkl', final_tr_with_all_data)
    
    dump_data_as_pickle('test_geno_with_8_m_avg_'+version+'.pkl', final_test)
    dump_data_as_pickle('test_pheno_with_8_m_avg_'+version+'.pkl', test_pheno)
    dump_data_as_pickle('test_geno_with_8_m_avg_all_data_'+version+'.pkl', final_test_with_all_data)
    
    dump_data_as_pickle('test_geno_unique_env_with_8_m_avg_'+version+'.pkl', final_test_unique_env)
    dump_data_as_pickle('test_pheno_uniqeu_env_with_8_m_avg_'+version+'.pkl', test_pheno_unique_env)
    dump_data_as_pickle('test_geno_unique_env_with_8_m_avg_all_data_'+version+'.pkl', final_test_with_all_data_unique_env)
    
    dump_data_as_pickle('val_geno_with_8_m_avg_'+version+'.pkl', final_val)
    dump_data_as_pickle('val_pheno_with_8_m_avg_'+version+'.pkl', val_pheno)
    dump_data_as_pickle('val_pheno_with_8_m_avg_all_data_' + version + '.pkl', final_val_with_all_data)

version:  v1
tr data shape:  (63707, 4133)
test data shape:  (12978, 4133)
test data unique env shape:  (23767, 4133)
val data shape:  (11384, 4133)
version:  v2
tr data shape:  (68589, 4133)
test data shape:  (15423, 4133)
test data unique env shape:  (16343, 4133)
val data shape:  (11843, 4133)
version:  v3
tr data shape:  (68704, 4133)
test data shape:  (14577, 4133)
test data unique env shape:  (17731, 4133)
val data shape:  (11309, 4133)
version:  v4
tr data shape:  (61250, 4133)
test data shape:  (13283, 4133)
test data unique env shape:  (25154, 4133)
val data shape:  (12041, 4133)
version:  v5
tr data shape:  (64802, 4133)
test data shape:  (13111, 4133)
test data unique env shape:  (21123, 4133)
val data shape:  (12780, 4133)


In [34]:
# tr_data = pd.read_pickle('../processed_data/feature_selected_training.pkl')
# final_tr, tr_pheno = create_dataset_with_env_data(tr_data,final_env_data)

In [35]:
# print('training data: ', final_tr.shape)
# print('pheno : ', tr_pheno.shape)

In [36]:
# test_data = pd.read_pickle('../processed_data/feature_selected_test.pkl')
# final_test, test_pheno = create_dataset_with_env_data(test_data,final_env_data)
# final_test_with_all_data = create_dataset_with_all_env_data(test_data, final_env_data)

In [37]:
# print('test data: ', final_test.shape)
# print('pheno : ', test_pheno.shape)

In [38]:
# val_data = pd.read_pickle('../processed_data/feature_selected_validation.pkl')
# final_val, val_pheno = create_dataset_with_env_data(val_data,final_env_data)

In [39]:
# print('validation data: ', final_val.shape)
# print('pheno : ', val_pheno.shape)

In [40]:
# dump_data_as_pickle('tr_geno_with_8_m_avg.pkl', final_tr)
# dump_data_as_pickle('tr_pheno_with_8_m_avg.pkl', tr_pheno)

In [41]:
# dump_data_as_pickle('test_geno_with_8_m_avg.pkl', final_test)
# dump_data_as_pickle('test_pheno_with_8_m_avg.pkl', test_pheno)

In [42]:
# dump_data_as_pickle('val_geno_with_8_m_avg.pkl', final_val)
# dump_data_as_pickle('val_pheno_with_8_m_avg.pkl', val_pheno)

In [43]:
# dump_data_as_pickle('test_geno_with_8_m_avg_all_data.pkl', final_test_with_all_data)

In [44]:
# final_test_with_all_data.head()